In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
path = 'D:/Work/DAandML/kaggle.com/ForestCoverType'

In [ ]:
train = pd.read_csv(path+'/train.csv')
train.head()

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
train.dtypes

In [ ]:
# Drop ID columns
train = train.iloc[:, 1:]
train.head(2)

In [ ]:
train.isnull().sum()

So there is all not null values

In [ ]:
# Select all features except 'Cover Type' and 'Id'
feature_cols = [col for col in train.columns if col not in ['Cover_Type','Id']]
feature_cols

In [ ]:
import seaborn as sns

In [ ]:
train.skew()

In [ ]:
train.std()

We see that all classes have an equal presence. No class re-balancing is necessary

In [ ]:
train.Cover_Type.value_counts()

Data Interaction and Visualization

In [ ]:
size_corr = 10 # Number of columns to calculate correlation
threshold = 0.5
list_corr = []

In [ ]:
data_corr = train.iloc[:,0:size_corr].corr()
print(data_corr, data_corr.shape)

In [ ]:
list_corr = []
# Filter the correlation
for i in range(0,size_corr):
    for j in range(i+1, size_corr):
        if((data_corr.iloc[i,j]  > threshold) and (data_corr.iloc[i,j] < 1) or ((data_corr.iloc[i,j] <0) and (data_corr.iloc[i,j]>-threshold))):
            list_corr.append([data_corr.iloc[i,j], i, j])

list_corr.sort()
(list_corr, len(list_corr))

In [ ]:
# print filtered correlation with column name
for corr, i, j in list_corr:
    print ('%s - %s: %.2f' % (train.columns[i], train.columns[j], corr))

Remove columns with Zero standard deviation

In [ ]:
train.std()

In [ ]:
cols = train.columns
remove_col = []

for col in cols:
    if (train[col].std() == 0):
        remove_col.append(col)
remove_col

In [ ]:
train = train.drop(remove_col, axis=1)
train.head()

In [ ]:
# First try with Random Forest

from sklearn.ensemble import RandomForestClassifier
row, col = train.shape
test.head()

row, col = train.shape
x_train = train.iloc[:,0:(col-1)]
y_train = train.iloc[:, (col-1)]

from sklearn.cross_validation import train_test_split
x,x_validate, y, y_validate = train_test_split(x_train, y_train)

clf = RandomForestClassifier(n_estimators=100)
clf.fit(x, y)
print(clf)

clf.score(x_validate, y_validate)



In [ ]:
test = pd.read_csv(path+'/test.csv')
result = test[['Id']]

test = test.drop('Id', axis=1)
test = test.drop(remove_col, axis=1)
test.head()

In [ ]:
x_test = test.iloc[:,0:col]
y_test = clf.predict(x_test)

import collections
len(y_test),collections.Counter(y_test)

result.loc[:,'Cover_Type'] = y_test
result.to_csv('Submission1.csv', index=False)

In [ ]:
x_test = test.iloc[:,0:col].values
y_test = clf.predict(x_test)

In [ ]:
row, col = train.shape
x_train = train.iloc[:,0:(col-1)]
y_train = train.iloc[:, (col-1)]

In [ ]:
# Second try, Random Forest Classifier with gridsearch
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
parameter_grid = {'n_estimators': [100, 150, 200, 300],
                  "max_depth": [3, None],
                  'criterion': ['gini', 'entropy'],
                  'max_features': ['auto', 'sqrt', 'log2']}

cross_validation = StratifiedKFold(y_train, n_folds=10)

grid_search = GridSearchCV(clf,
                           param_grid=parameter_grid,
                           cv=cross_validation)

grid_search.fit(x_train, y_train)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

grid_search.best_estimator_

In [ ]:
test = pd.read_csv(path+'/test.csv')
result = test[['Id']]

test = test.drop('Id', axis=1)
test = test.drop(remove_col, axis=1)
x_test = test.iloc[:,0:col].values

In [ ]:
clf_grid = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
clf_grid.score(x_train, y_train)

In [ ]:
y_test = clf_grid.predict(x_test)
result.loc[:,'Cover_Type'] = y_test
result.to_csv('Submission2.csv', index=False)